#### Eksik Verileri Tamamlama

Artık eksik değerlerle çalışma konusunda bir miktar deneyiminiz var ve ortak yöntemlere dayalı olarak eksik değerleri doldurmak konusunda deneyim kazandınız. Şimdi sıra sizde, NaN değerleri olan satırlar için bile tahmin yapabilme becerilerinizi kullanmaya geldi.

İlk olarak, gerekli kütüphaneleri içe aktaralım ve önceki denemenizde elde ettiğiniz sonuçları bir araya getirelim.

Aşağıdaki kod parçası, veri analizi ve makine öğrenimi işlemleri gerçekleştirmek amacıyla gerekli olan kütüphaneleri içe aktarıyor.
* numpy kütüphanesi sayısal hesaplamalar yapmak için,
* pandas veri çerçeveleri oluşturmak ve veri manipülasyonu yapmak için,
* matplotlib ve seaborn veri görselleştirmesi için kullanılırken,
* LinearRegression makine öğrenimi modeli için, train_test_split veriyi eğitim ve test kümelerine bölmek için,
* r2_score ve mean_squared_error ise model performansını değerlendirmek için kullanılır.
* %matplotlib inline komutu sayesinde görselleştirmelerin Jupyter Notebook'ta görüntülenmesini sağlar.

* Kodun devamında, ./survey_results_public.csv dosyasından veriler okunur ve df adında bir veri çerçevesine yüklenir.
* df.head() komutuyla bu veri çerçevesinin ilk birkaç satırı görüntülenir, bu sayede verinin yapısı hakkında hızlı bir fikir edinilebilir.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import ImputingValues as t
import seaborn as sns
%matplotlib inline

df = pd.read_csv('./survey_results_public.csv')
df.head()



,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Bu kod parçası, veri setinde yalnızca nicel değişkenleri kullanmayı ve eksik değerleri içeren satırları çıkarmayı amaçlar.
* İlk olarak, 'Salary', 'CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction' ve 'StackOverflowSatisfaction' gibi nicel değişkenlerden oluşan bir alt veri çerçevesi (num_vars) oluşturulur ve eksik değerleri içeren satırlar çıkarılarak (dropna) temiz bir alt veri çerçevesi (df_dropna) elde edilir.

* Daha sonra, bu temiz veri çerçevesi üzerinden bağımsız ve bağımlı değişkenler ayrıştırılır. Bağımsız değişkenler (X), 'CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction' ve 'StackOverflowSatisfaction' sütunlarından oluşurken, bağımlı değişken (y) 'Salary' sütununu içerir.

* Veri daha sonra eğitim ve test kümelerine bölünür. train_test_split fonksiyonu kullanılarak, verinin %70'i eğitim kümesine, %30'u ise test kümesine atanır. Bu bölünme işlemi, modelin eğitimi ve performans değerlendirmesi için gereklidir.

* Daha sonra, normalize edilmiş bir doğrusal regresyon modeli (LinearRegression) oluşturulur (lm_model), eğitim kümesi (X_train ve y_train) üzerinde uydurulur (fit).
* Model eğitildikten sonra, test kümesi (X_test) üzerinde tahminler yapılır (predict) ve modelin performansı ölçülür.
* Elde edilen tahminlerle gerçek değerler arasındaki uyum, r-kare skoru (r2_score) ile değerlendirilir.
* Sonuç, r-kare skoru ve test kümesindeki veri noktalarının sayısı ile birlikte ekrana yazdırılır. Bu kod parçası, veri seti üzerinde temizlik, bölme, model oluşturma ve değerlendirme adımlarını içerir.

In [ ]:
# Sadece nicel değişkenleri kullanın ve eksik değeri olan satırları çıkarın

num_vars = df[['Salary', 'CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction', 'StackOverflowSatisfaction']]
# Eksik değer içeren satırları çıkar
df_dropna = num_vars.dropna(axis=0)  # Eksik değer içeren satırları çıkar

# Bağımsız ve bağımlı değişkenlere ayır
X = df_dropna[['CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction', 'StackOverflowSatisfaction']]  # Bağımsız değişkenler
y = df_dropna['Salary']  # Bağımlı değişken

# Eğitim ve test kümelerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)  # Veriyi eğitim ve test kümelerine bölelim

# Modeli oluştur
lm_model = LinearRegression(normalize=True)

# Modeli eğit
lm_model.fit(X_train, y_train)

# Modeli kullanarak tahmin yap ve sonuçları değerlendir
 # Test verileri üzerinde tahmin yap
y_test_preds = lm_model.predict(X_test)
"Modelinizin r-kare skoru {} ve {} değeri üzerinde hesaplandı.".format(r2_score(y_test, y_test_preds), len(y_test))


'Modelinizin r-kare skoru 0.01917066180376248 ve 645 değeri üzerinde hesaplandı.'

Aşağıdaki kod parçası, veri çerçevesindeki eksik maaş değerlerine sahip satırları çıkarmayı amaçlayan bir işlemi içerir. İşlemler aşağıdaki adımlarla gerçekleştirilir:

**drop_sal_df = num_vars.dropna(subset=['Salary'], axis=0):** Bu satırda, num_vars adlı veri çerçevesinden 'Salary' (Maaş) değişkeninde eksik değere sahip olan satırlar çıkarılır. dropna fonksiyonu, eksik değer içeren satırları çıkarmak için kullanılır. subset=['Salary'] parametresi, yalnızca 'Salary' değişkeninde eksik değeri olan satırların seçilmesini sağlar. axis=0 parametresi, işlemin satır bazında yapılacağını belirtir.

**drop_sal_df.head():** Bu satır, işlem sonrasında oluşturulan yeni veri çerçevesinin başlangıçtaki birkaç satırını görüntüler. Bu, işlemin doğru bir şekilde gerçekleştirildiğini kontrol etmek amacıyla kullanılır.

Bu kod parçası, 'Salary' (Maaş) değişkenindeki eksik değerlere sahip satırları çıkararak veri setinin temizlenmesini sağlar. Böylece, analizlerin veya model eğitimlerinin daha güvenilir ve tutarlı sonuçlar üretmesi amaçlanır.

In [ ]:
drop_sal_df = num_vars.dropna(subset=['Salary'], axis=0) #Drop the rows with missing salaries
drop_sal_df.head()

,Salary,CareerSatisfaction,HoursPerWeek,JobSatisfaction,StackOverflowSatisfaction
2,113750.0,8.0,NaN,9.0,8.0
14,100000.0,8.0,NaN,8.0,8.0
17,130000.0,9.0,NaN,8.0,8.0
18,82500.0,5.0,NaN,3.0,NaN
22,100764.0,8.0,NaN,9.0,8.0


Aşağıdaki kod, eksik değerleri sütun ortalamasıyla dolduran bir işlemi gerçekleştirir. İlk kod parçasında, her sütun için ortalama hesaplanır ve eksik değerler bu ortalamalarla doldurulur. Elde edilen yeni veri çerçevesi, eksik değerlerden arındırılmış bir versiyonu temsil eder. fill_df.head() satırı, işlemin doğruluğunu kontrol etmek için oluşturulan yeni veri çerçevesinin ilk birkaç satırını görüntüler.

In [ ]:
fill_mean = lambda col: col.fillna(col.mean()) # Ortalama fonksiyonu

fill_df = drop_sal_df.apply(fill_mean, axis=0) # Eksik değerleri sütun ortalamalarıyla doldur.

# Test görünümü
fill_df.head()

,Salary,CareerSatisfaction,HoursPerWeek,JobSatisfaction,StackOverflowSatisfaction
2,113750.0,8.0,2.447415,9.0,8.000000
14,100000.0,8.0,2.447415,8.0,8.000000
17,130000.0,9.0,2.447415,8.0,8.000000
18,82500.0,5.0,2.447415,3.0,8.442686
22,100764.0,8.0,2.447415,9.0,8.000000


Bu kod parçası, bir doğrusal regresyon modelini oluşturmak, eğitmek, tahmin yapmak ve sonuçları değerlendirmek amacıyla kullanılan işlemleri içerir:

* Veri seti bağımsız değişkenler (X) ve bağımlı değişken (y) olarak ayrıştırılır.
* Veri seti, eğitim ve test kümelerine bölünür. Veri setinin %30'u test kümesi olarak ayrılır.
* LinearRegression sınıfı kullanılarak bir doğrusal regresyon modeli (lm_model) oluşturulur. normalize=True parametresi, bağımsız değişkenleri normalize etmek için kullanılır.
* Oluşturulan model, eğitim verileriyle (X_train, y_train) eğitilir.
* Model, test verileri üzerinde tahmin yaparak y_test_preds tahminlerini oluşturur.
* r2_score fonksiyonu kullanılarak modelin R-kare skoru (rsquared_score) hesaplanır.
* Test kümesinin uzunluğu (length_y_test) hesaplanır.
* Sonuç mesajı, elde edilen R-kare skoru ve test kümesinin uzunluğu kullanılarak oluşturulur.
##### Bu kod parçası, veri analizi ve model performans değerlendirmesi için önemli bir süreci temsil eder.

In [ ]:
# Veri setini bağımlı ve bağımsız değişkenlere ayır
X = fill_df[['CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction', 'StackOverflowSatisfaction']]
y = fill_df['Salary']

# Veriyi eğitim ve test kümelerine bölelim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=42)

# Doğrusal Regresyon modelini oluştur (nesne oluşturma)
lm_model = LinearRegression(normalize=True)

# Modeli eğit
lm_model.fit(X_train, y_train)

# Modeli kullanarak tahmin yap
y_test_preds = lm_model.predict(X_test)

# R-kare skoru hesapla
rsquared_score = r2_score(y_test, y_test_preds)

# Test kümesinin uzunluğunu al
length_y_test = len(y_test)

# Sonuç mesajını oluştur ve formatla
"Modelinizin r-kare skoru {} ve {} değeri üzerinde hesaplandı.".format(rsquared_score, length_y_test)


'Modelinizin r-kare skoru 0.03257139063404435 ve 1503 değeri üzerinde hesaplandı.'